<a href="https://colab.research.google.com/github/antgr/pytorch-nli/blob/master/1d%20Simple%20NLI%20Model%20with%20transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import random
import numpy as np

import time


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', lower = True)
LABEL = data.LabelField()

In [2]:
train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [00:25<00:00, 3.66MB/s]


extracting


In [3]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:37, 2.17MB/s]                          
100%|█████████▉| 399979/400000 [00:51<00:00, 7945.78it/s]

In [0]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
class NLIRNN(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim,
                 hidden_dim,
                 n_encoder,
                 n_head,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_dim,
                          batch_first=True,
                          bidirectional=False)   
        
        self.transformerEncoderLayer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=n_head)
        self.transformerEncoder = nn.TransformerEncoder(self.transformerEncoderLayer, num_layers=n_encoder)

        self.dropout = nn.Dropout(p=dropout)

        self.fc1 = nn.Linear(in_features=hidden_dim,
                             out_features=hidden_dim)
        self.fc2 = nn.Linear(in_features=hidden_dim,
                             out_features=output_dim)

        
    def forward(self, prem, hypo):
        embedded_prem = self.embedding(prem)
        print("embedded_prem: ", embedded_prem.shape)
        embedded_hypo = self.embedding(hypo)
        print("embedded_hypo: ", embedded_hypo.shape)

        #embedded_prem = [sent len1, batch size, hid dim]
        #embedded_hypo = [sent len2, batch size, hid dim]

        out_prem_gru, _ = self.gru(embedded_prem)
        print("out_prem_gru: ", out_prem_gru.shape)
        out_hypo_gru, _ = self.gru(embedded_hypo)
        print("out_hypo_gru: ", out_hypo_gru.shape)

        #out_prem_gru = [sent len1, batch size, hid dim]
        #out_prem_gru = [sent len2, batch size, hid dim]

        hidden = torch.cat((out_prem_gru, out_hypo_gru), dim=0)

        encoded_inputs = self.transformerEncoder(hidden)
        print("encoded_inputs: ", encoded_inputs.shape)

        #encoded_inputs = [sent len3, batch size, hid dim]

        out = self.dropout(F.relu(self.fc1(encoded_inputs)))

        #out:  = [sent len3, batch size, hidden]

        print("out: ", out.shape)

        prediction = self.fc2(out)

        #out:  = [sent len3, batch size, output_dim]

        print("prediction: ", prediction.shape)
        prediction = prediction.permute(1,0,2)

        #out:  = [batch size, sent len3, output_dim]

        prediction = prediction.sum(dim = 1)

        #out:  = [batch size, output_dim]

        print("prediction: ", prediction.shape)
        return prediction

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
#FC_LAYERS = 3
N_ENCODER = 2
N_HEADS = 4
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = NLIRNN(INPUT_DIM,
               EMBEDDING_DIM,
               HIDDEN_DIM,
               N_ENCODER,
               N_HEADS,
               OUTPUT_DIM,
               DROPOUT,
               PAD_IDX)

In [7]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1)
        
model.apply(init_weights)

NLIRNN(
  (embedding): Embedding(23566, 300, padding_idx=1)
  (gru): GRU(300, 300, batch_first=True)
  (transformerEncoderLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=300, out_features=300, bias=True)
    )
    (linear1): Linear(in_features=300, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=300, bias=True)
    (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformerEncoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=2048, bias=True)
        (drop

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,483,447 trainable parameters


In [9]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([23566, 300])


In [10]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.2983, -0.2664, -0.0631,  ..., -0.1577,  1.0438, -0.6433],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.2983, -0.2664, -0.0631,  ..., -0.1577,  1.0438, -0.6433],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])


In [0]:
model.embedding.weight.requires_grad = False

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,413,647 trainable parameters


In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss()

In [16]:
model = model.to(device)
criterion = criterion.to(device)

100%|█████████▉| 399979/400000 [01:10<00:00, 7945.78it/s]

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        prem = batch.premise
        hypo = batch.hypothesis
        labels = batch.label
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

embedded_prem:  torch.Size([67, 512, 300])
embedded_hypo:  torch.Size([23, 512, 300])
out_prem_gru:  torch.Size([67, 512, 300])
out_hypo_gru:  torch.Size([23, 512, 300])
encoded_inputs:  torch.Size([90, 512, 300])
out:  torch.Size([90, 512, 300])
prediction:  torch.Size([90, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([43, 512, 300])
embedded_hypo:  torch.Size([35, 512, 300])
out_prem_gru:  torch.Size([43, 512, 300])
out_hypo_gru:  torch.Size([35, 512, 300])
encoded_inputs:  torch.Size([78, 512, 300])
out:  torch.Size([78, 512, 300])
prediction:  torch.Size([78, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([41, 512, 300])
embedded_hypo:  torch.Size([23, 512, 300])
out_prem_gru:  torch.Size([41, 512, 300])
out_hypo_gru:  torch.Size([23, 512, 300])
encoded_inputs:  torch.Size([64, 512, 300])
out:  torch.Size([64, 512, 300])
prediction:  torch.Size([64, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([56, 512, 300])


In [22]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

embedded_prem:  torch.Size([7, 512, 300])
embedded_hypo:  torch.Size([11, 512, 300])
out_prem_gru:  torch.Size([7, 512, 300])
out_hypo_gru:  torch.Size([11, 512, 300])
encoded_inputs:  torch.Size([18, 512, 300])
out:  torch.Size([18, 512, 300])
prediction:  torch.Size([18, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([11, 512, 300])
embedded_hypo:  torch.Size([15, 512, 300])
out_prem_gru:  torch.Size([11, 512, 300])
out_hypo_gru:  torch.Size([15, 512, 300])
encoded_inputs:  torch.Size([26, 512, 300])
out:  torch.Size([26, 512, 300])
prediction:  torch.Size([26, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([11, 512, 300])
embedded_hypo:  torch.Size([7, 512, 300])
out_prem_gru:  torch.Size([11, 512, 300])
out_hypo_gru:  torch.Size([7, 512, 300])
encoded_inputs:  torch.Size([18, 512, 300])
out:  torch.Size([18, 512, 300])
prediction:  torch.Size([18, 512, 3])
prediction:  torch.Size([512, 3])
embedded_prem:  torch.Size([15, 512, 300])
embe

In [0]:
#Test Loss: 0.697 |  Test Acc: 71.72%